In [8]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from extract import LOCM2
from utils import read_plan, read_json_file
from evaluator import ExecutabilityEvaluator
from collections import defaultdict
import json

In [9]:
def test_cross_exe(domain_name, train_traces , prefix_traces, gt_traces=[],gen_seq_rate=None):
    debug = {}
    try:
        polocm2 = LOCM2(state_param=True, viz=False, debug=debug)
        model = polocm2.extract_model(train_traces)
        learned_domain = model.to_pddl_domain(domain_name)
        gt_filename = os.path.join('../../data', 'goose-benchmarks', 'tasks', domain_name, 'domain.pddl')
        
        evaluator = ExecutabilityEvaluator(learned_domain,gt_filename, debug=False)

        exe = evaluator.get_cross_executability(prefix_traces, gt_traces, gen_seq_rate)
        return exe
    except Exception as e:
        print(f"Error processing domain {domain_name}: {e}")
        return 0,0
    
def test_balanced_exe(domain_name,train_traces, test_traces, test_invalid_traces, binary=False):
    debug = {}
    try:
        polocm2 = LOCM2(state_param=True, viz=False, debug=debug)
        model = polocm2.extract_model(train_traces)
        learned_domain = model.to_pddl_domain(domain_name)
        gt_filename = os.path.join('../../data', 'goose-benchmarks', 'tasks', domain_name, 'domain.pddl')
        
        evaluator = ExecutabilityEvaluator(learned_domain, gt_filename, debug=False)
        exe = evaluator.get_balanced_executability(test_traces, test_invalid_traces, binary)

        return exe
    except Exception as e:
        print(f"Error processing domain {domain_name}: {e}")
        return 0,0

In [10]:
plain_traces = defaultdict(list)
with open("../../data/plain_traces/plain_traces.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        details = line.split("&&")

        domain_name = details[0]
        plan = details[-1]

        plain_traces[domain_name].append(read_plan(plan))

In [11]:
train_traces = defaultdict(list)
data = read_json_file("../../data/training_data/traces_plan_r5.json")
for learning_obj in data:
    domain_name = learning_obj["domain"]
    train_traces[domain_name].append(learning_obj)
    


In [12]:
valid_traces = defaultdict(list)
invalid_traces = defaultdict(list)

with open("../../data/plain_traces/random_walks.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        details = line.split("&&")
        domain_name = details[0]
        is_valid = details[1] == "rand"
        plan = details[-1]
        if is_valid:
            valid_traces[domain_name].append(read_plan(plan))
        else:
            invalid_traces[domain_name].append(read_plan(plan))

In [13]:
# with open("./cross_exe_results.csv", "w") as f:
#     f.write("ID, Domain, len, len%, l_exe, gt_exe\n")
#     for domain, items in train_traces.items():
#         print(f"Testing domain: {domain}")
#         for item in items:
#             traces = item["traces"]

#             try:
#                 coress_exe = test_cross_exe(domain, traces, plain_traces[domain], plain_traces[domain])
#             except Exception as e:
#                 print(f"Error in balanced executability for domain {domain}: {e}")
#                 coress_exe = 0,0

#         # Test balanced executability
#             f.write(f"{item['id']}, {domain}, {item['total_length']}, {item['len%']}, {coress_exe[0]}, {coress_exe[1]}\n")
#             f.flush()
#             print(f"Cross Executability for {domain}: {coress_exe}")

In [ ]:
with open("./cross_exe_results.csv", "w") as f:
    f.write("ID, Domain, len%, objs, gen_seq_rate, l_exe, gt_exe\n")
    for domain, items in train_traces.items():
        print(f"Testing domain: {domain}")
        for item in items:
            traces = item["traces"]
            for i in range(1, 10):
                gen_seq_rate = i / 10
                try:
                    coress_exe = test_cross_exe(domain, traces, plain_traces[domain], plain_traces[domain], gen_seq_rate=gen_seq_rate)
                except Exception as e:
                    print(f"Error in cross executability for domain {domain}: {e}")
                    coress_exe = 0,0

            # Test balanced executability
                f.write(f"{item['id']}, {domain}, {item['total_length']}, {item['len%']}, {gen_seq_rate}, {coress_exe[0]}, {coress_exe[1]}\n")
                f.flush()
                print(f"Cross Executability for {domain}: {coress_exe}")

Testing domain: blocksworld
Cross Executability for blocksworld: (0.5051855345627754, 1.0)
Cross Executability for blocksworld: (0.5051855345627754, 1.0)
Cross Executability for blocksworld: (0.5051855345627754, 0.9894736842105263)
Cross Executability for blocksworld: (0.5051855345627754, 0.9894736842105263)
Cross Executability for blocksworld: (0.5051855345627754, 0.9696969696969697)
Cross Executability for blocksworld: (0.5051855345627754, 0.9696969696969697)
Cross Executability for blocksworld: (0.5051855345627754, 0.9696969696969697)
Cross Executability for blocksworld: (0.5051855345627754, 0.9696969696969697)
Cross Executability for blocksworld: (0.5051855345627754, 0.9696969696969697)
Cross Executability for blocksworld: (0.7706344335475591, 0.8435920197156151)
Cross Executability for blocksworld: (0.7706344335475591, 0.8853757751607214)
Cross Executability for blocksworld: (0.7706344335475591, 0.9107612465507203)
Cross Executability for blocksworld: (0.7706344335475591, 0.935623

In [18]:

# with open("./balanced_exe_results.csv", "w") as f:
#     f.write("ID, Domain, len, objs, valid_exe, invalid_exe\n")
#     for domain, items in train_traces.items():
#         print(f"Testing domain: {domain}")
#         for item in items:
#             traces = item["traces"]

#             try:
#                 balanced_exe = test_balanced_exe(domain, traces, valid_traces[domain], invalid_traces[domain])
#             except Exception as e:
#                 print(f"Error in balanced executability for domain {domain}: {e}")
#                 balanced_exe = 0,0

#         # Test balanced executability
#             f.write(f"{item['id']}, {domain}, {item['total_length']}, {item['number_of_objects']}, {balanced_exe[0]}, {balanced_exe[1]}\n")
#             f.flush()
#             print(f"Balanced Executability for {domain}: {balanced_exe}")

In [ ]:
# with open("./balanced_binary_exe_results.csv", "w") as f:
#     f.write("ID, Domain, len, len%, valid_exe, invalid_exe\n")
#     for domain, items in train_traces.items():
#         print(f"Testing domain: {domain}")
#         for item in items:
#             traces = item["traces"]

#             try:
#                 balanced_exe = test_balanced_exe(domain, traces, valid_traces[domain], invalid_traces[domain], binary=True)
#             except Exception as e:
#                 print(f"Error in balanced executability for domain {domain}: {e}")
#                 balanced_exe = 0,0

#         # Test balanced executability
#             f.write(f"{item['id']}, {domain}, {item['total_length']}, {item['len%']}, {balanced_exe[0]}, {balanced_exe[1]}\n")
#             f.flush()
#             print(f"Balanced Executability for {domain}: {balanced_exe}")

Testing domain: blocksworld
Balanced Executability for blocksworld: (0.19686526926633768, 0.9986531986531986)
Balanced Executability for blocksworld: (0.28513642468396816, 0.9885521885521885)
Balanced Executability for blocksworld: (0.19686526926633768, 0.9986531986531986)
Balanced Executability for blocksworld: (0.19686526926633768, 0.9986531986531986)
Balanced Executability for blocksworld: (0.19686526926633768, 0.9986531986531986)
Balanced Executability for blocksworld: (0.19686526926633768, 0.9986531986531986)
Balanced Executability for blocksworld: (0.28044776143401734, 0.9837037037037037)
Balanced Executability for blocksworld: (0.28044776143401734, 0.9837037037037037)
Balanced Executability for blocksworld: (0.28044776143401734, 0.9837037037037037)
Balanced Executability for blocksworld: (0.19686526926633768, 0.9986531986531986)
Balanced Executability for blocksworld: (0.19686526926633768, 0.9986531986531986)
Balanced Executability for blocksworld: (0.19686526926633768, 0.998653

In [ ]:
# with open("../../data/plain_traces/plain_traces.txt", "r") as f:
#     with open("res.csv", "w") as out:
#         lines = f.readlines()
#         for line in lines:
#             try:
#                 details = line.split("&&")
#                 name = f"{details[0]}-{details[2]}-{details[3]}"
            
#                 plan = details[-1]
#                 trace = read_plan(plan)
           

#                 domain_name = details[0]
            
#                 exe_l, exe_gt = test_cross_exe(domain_name, trace, trace)
#             except Exception as e:
#                 print(f"Error processing task {name}: {e}")
#                 exe_l, exe_gt = 0, 0
#             print(f"Executability for {name}: {exe_l}, {exe_gt}")
#             out.write(f"{name},{exe_l}, {exe_gt}\n")
#             out.flush()